This notebook provides code snippets to identify and remove disconnect (and potentially motion) related artifacts in your recordings so that they can be ignored during post-processing.

In [1]:
# Import all the functions you need
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

# Comment here to add in for either mac or linux computer
# sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])  # For mac laptop
sys.path.extend(['/data/GitHub/NeuroPy'])  # for linux

from neuropy import core
from neuropy.io import (optitrackio,
                        dlcio,
                        )
from neuropy.io.neuroscopeio import NeuroscopeIO
from neuropy.io.binarysignalio import BinarysignalIO 
from neuropy.io.miniscopeio import MiniscopeIO
from neuropy.core.session import ProcessData
from neuropy.core.signal import Signal

plt.rcParams['pdf.fonttype'] = 42


In [2]:
base_dir = Path("/data/Clustering/sessions/RatU/RatUNSD_sleep_score_test")
sess = ProcessData(base_dir)
print(sess.recinfo)

filename: /data/Clustering/sessions/RatU/RatUNSD_sleep_score_test/RatU_Day2NSD_2021-07-24_08-16-38.theta_sw_ch_only.xml 
# channels: 2
sampling rate: 30000
lfp Srate (downsampled): 1250



In [3]:
print(sess.eegfile)

duration: 48498.53 seconds 
duration: 13.47 hours 



## Wrangle EEG and EMG data into AccuSleePy format

### Load in EMG and EEG data

In [4]:
eeg = sess.eegfile.get_signal(0)  # grab from channel 0, in this case the electrode in the pyramidal cell layer
emg = Signal.from_dict(np.load(sorted(base_dir.glob("*.emg.npy"))[0], allow_pickle=True).item())

### Interpoloate EMG to match eeg data

In [5]:
emg_interp = np.interp(eeg.time, emg.time, emg.traces[0])

### Convert into DataFrame

In [6]:
accusleepy_df = pd.DataFrame({"eeg": eeg.traces[0], "emg": emg_interp})
accusleepy_df

,eeg,emg
0,207,0.544728
1,569,0.544728
2,506,0.544728
3,510,0.544728
4,721,0.544728
...,...,...
60623162,-48,0.411182
60623163,397,0.411494
60623164,522,0.411805
60623165,362,0.412117


### Save as CSV file - slow and creates a huge file

In [30]:
accusleepy_df.to_csv(sess.basepath / "accusleepy_eeg_emg.csv")

### Save as parquet file: fast and creates smaller file but requires additional packages
`conda install -c conda-forge pyarrow` or
`conda install -c conda-forge fastparquet`

See: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html

In [7]:
accusleepy_df.to_parquet(sess.basepath / "accusleepy_eeg_emg.parquet")